## Import Packages

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_validate, KFold
import joblib

In [2]:
df = pd.read_csv('../data/interim/2_data_engineered.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   HR_before            16 non-null     float64
 1   HR_after             16 non-null     float64
 2   age                  16 non-null     float64
 3   height               16 non-null     float64
 4   weight               16 non-null     float64
 5   sex                  16 non-null     float64
 6   sp_total_distance    16 non-null     float64
 7   sp_average_distance  16 non-null     float64
 8   sp_median_distance   16 non-null     float64
 9   sp_total_angle       16 non-null     float64
 10  sp_average_angle     16 non-null     float64
 11  sp_median_angle      16 non-null     float64
 12  row_id               16 non-null     float64
 13  total_latency        16 non-null     float64
 14  mean_latency         16 non-null     float64
 15  median_latency       16 non-null     float

In [4]:
df = df.dropna()

## Choose your Feature!

In [5]:
df = df[['HR_before','vo2_max','sp_average_distance','sp_average_angle','mean_latency']]
# df = df[['HR_before','vo2_max','sp_average_distance','sp_average_angle','mean_latency']]

## Train Test Split

In [6]:
# Assuming 'df' is your DataFrame and 'target_column' is the name of the target variable
X = df.drop(columns=['vo2_max'])  # Features (drop the target column)
y = df['vo2_max']                 # Target (the column you're predicting)

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
X

,HR_before,sp_average_distance,sp_average_angle,mean_latency
0,72.0,33.427300,0.098395,819.000000
1,84.0,27.215296,0.170527,496.000000
2,94.0,36.672403,0.109743,671.473684
3,83.0,23.899725,0.084702,573.238095
4,70.0,23.528027,0.096061,645.600000
5,68.0,24.317028,0.110522,614.826087
6,68.0,28.528652,0.078891,677.818182
7,78.0,28.951215,0.097220,673.571429
8,85.0,45.482336,0.171649,651.833333
9,96.0,33.055179,0.148485,776.611111


## Define Evaluation Metrics Function

In [8]:
def evaluate_model(model, model_name, X, y):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    # Perform 5-fold cross-validation
    cv_scores = cross_validate(model, X, y, cv=kfold, scoring=['neg_root_mean_squared_error', 'r2', 'neg_mean_absolute_percentage_error'])

    # Extracting individual metrics from the results
    rmse_scores = -cv_scores['test_neg_root_mean_squared_error']  # Negate to make positive
    # r2_scores = cv_scores['test_r2']
    mape_scores = -cv_scores['test_neg_mean_absolute_percentage_error']  # Negate to make positive

    # return [model_name, round(float(rmse_scores.mean()),5), round(float(r2_scores.mean()),5), round(float(mape_scores.mean())*100,5)]
    return [model_name, round(float(rmse_scores.mean()),5), round(float(mape_scores.mean())*100,5)]

## Modelling

### Linear Regression

In [9]:
# Initialize the Linear Regression model
lr = LinearRegression()

# Train the model on the training data
lr.fit(X_train, y_train)

# Make predictions on the test data
y_pred_lr = lr.predict(X_test)

lr_scores = evaluate_model(lr, 'Linear Regression', X, y)

## Random Forest

In [10]:
rf = RandomForestRegressor(random_state=42)

# rf.fit(X_train, y_train)

# y_pred_lr = rf.predict(X_test)

rf_scores = evaluate_model(rf, 'Random Forest Regressor', X, y)

## XGBoost

In [11]:
# Initialize XGBoost model
xgb_model = xgb.XGBRegressor(random_state=42)

xgb_scores = evaluate_model(xgb_model, 'XGBoost Model', X, y)

## Put into dataframe

In [12]:
scores = [lr_scores, rf_scores, xgb_scores]
scores = pd.DataFrame(scores, columns=['Model Name', 'RMSE', 'MAPE'])
scores

,Model Name,RMSE,MAPE
0,Linear Regression,2.89406,5.47193
1,Random Forest Regressor,2.71253,4.95316
2,XGBoost Model,2.92205,5.58402


## Save model into dataframe

Since the best model is linear regression, we will save linreg model

In [13]:
# Initialize the Linear Regression model
lr = RandomForestRegressor()

# Train the model on the training data
lr.fit(X, y)

# Get the current timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Assuming 'model' is your trained model
joblib.dump(lr, f'../models/random_forest_{timestamp}.pkl')

['../models/random_forest_20241107_104222.pkl']